# Scripts for ChroMo

This notebook includes the essential programs for processing and merging the files generated by the Fiji segmentation macro. The resulting files are specifically designed for ChroMo image analysis.

The script incorporates a linear velocity column and, if desired, performs time normalization. Additionally, it includes the necessary script to generate the horsetail and stop values files based on the linear velocity column.

### Script to process, merge and extract the horsetail movement stage from the CSV files generated by the Fiji macro

In [2]:
from os import listdir
from os.path import join, basename
from tkinter import Tk, Label, Entry, Button, filedialog, font, Text, BooleanVar, Checkbutton
from pandas import read_csv, concat, DataFrame
from pandas.errors import ParserError
from math import sqrt
from glob import glob
from configparser import ConfigParser


def process_for_ChroMo():
    '''
    Initializes the program to process, merge and generate the horsetail and stop files. 
    '''
    def browse_folder_path():
        folder_path = filedialog.askdirectory()
        path_entry.delete(0, 'end')
        path_entry.insert('end', folder_path)

    def browse_folder_save():
        folder_path = filedialog.askdirectory()
        save_path_entry.delete(0, 'end')
        save_path_entry.insert('end', folder_path)

    # Create an instance of ConfigParser
    config = ConfigParser()

    # Load the configuration file
    config.read('last_arguments_process.ini')

    def process_files(path, save_path, var_label, pattern, cal_speed,
                      normalize):
        '''
        Processes the files generated by the Fiji macro. It detects the frame where the cell is dividing and 
        changes the columns so that ChroMo can detect each parameter. 

        Arguments:
        - path (string): the path where the files generated by the Fiji macro are located. 
        - save_path (string): the path where you want to save the results. 
        - var_label (string): the name that you want to associate with the cell group, by default the strain name.
        - pattern (string): the pattern used to identify the files to process. }
        - cal_speed (bool): if checked (changing the default value to True), the function calculates linear velocity.
        - normalize (bool): if checked (changing the default value to True), the function does time normalization.
        '''

        pattern = pattern.lower()

        files = glob(join(path, f'*{pattern}*'))

        result_text.configure(state="normal")
        result_text.delete('1.0', 'end')  # Clear the previous results

        # Frame of division check
        for file in files:
            try:
                df = read_csv(file)
            except ParserError:
                result_text.insert('end', f"Error reading file: {file}\n")
                continue

            frame_delete = df[df['Label'].str.contains(
                r'[0-9]{4}-0002-[0-9]{4}')].index

            if len(frame_delete) > 0:
                result_text.insert(
                    'end', f"First frame deleted: {frame_delete[0]}\n")
                df = df.loc[:frame_delete[0] - 1]
            else:
                result_text.insert('end', "No matching rows found.\n")

            save_file = join(save_path,
                             basename(file).replace(".csv", "_processed.csv"))
            df.to_csv(save_file, index=False)

        files = glob(join(save_path, f'*{pattern}*'))

        result_text.insert('end', "\nFiles saved in the folder:\n")
        result_text.insert('end', "\n".join(basename(file) for file in files))

        # Calculates linear velocity used below
        def vector_magnitude(x1, y1, x2, y2):
            dx = x2 - x1
            dy = y2 - y1
            magnitude = sqrt(dx**2 + dy**2)
            return magnitude

        # This code changes the column names so ChroMo can automatically detect each parameter.
        for file in files:
            x = read_csv(file)
            x.columns = [
                "frame", "particle", "area", "x", "y", "XM", "YM", "perimeter",
                "major", "minor", "Angle", "circularity", "frame.1", "AR",
                "Roundness", "convexity"
            ]
            x['label'] = var_label
            x['particle'] = x['particle'].str.split("_R3D").str[0][0]
            x_subset = x.iloc[:, [0, 1, 2, 3, 4, 8, 9, 11, 15, 16]].copy()
            save_file = join(save_path, basename(file))
            x_subset.to_csv(save_file, index=False)

            # this code adds the column 'cal.speed' with the values of linear velocity of each frame
            if cal_speed == True:
                x1, y1, x2, y2 = None, None, None, None
                for index, row in x_subset.iterrows():
                    if index > 0:
                        if x2 is None:
                            x2 = row['x']
                            y2 = row['y']
                        else:
                            x1 = x2
                            y1 = y2
                            x2 = row['x']
                            y2 = row['y']
                            magnitude = vector_magnitude(x1, y1, x2, y2)
                            x_subset.loc[
                                index - 1,
                                'cal.speed'] = magnitude  # Assign the magnitude to the previous row

                x_subset = x_subset.iloc[1:
                                         -1]  # Remove the first and last rows

                save_file = join(save_path, basename(file))
                x_subset.to_csv(save_file, index=False)

            # Time normalization
            if normalize == True:
                x_subset['frame'] = (range(-1, -len(x_subset['frame']) - 1,
                                           -1))
                x_subset['frame'] = x_subset['frame'].sort_values().values

                x_subset.to_csv(save_file, index=False)

        # Save the arguments to the configuration file
        config['LastArgs'] = {
            'path': path,
            'save_path': save_path,
            'var_label': var_label,
            'pattern': pattern
        }
        with open('last_arguments_process.ini', 'w') as configfile:
            config.write(configfile)

        result_text.configure(
            state="disabled"
        )  #so that the user cannot edit the results window.

    def merge_files(
        path_processed_files,
        csv_name,
        pattern,
    ):
        '''
        Merges the processed files generated by the process_files program. 

        Arguments:
        - path_processed_files (string): the path where the processed files are and where the merged file is saved. 
        - csv_name (string): the name you want to assign to the merged file.
        - pattern (string): the pattern used to identify the files to merge. 
        '''

        result_text_merge.configure(state='normal')
        result_text_merge.delete('1.0', 'end')  # Clear the previous results

        files = [
            f for f in listdir(path_processed_files)
            if f.lower().endswith(".csv") and pattern.lower() in f.lower()
        ]

        total = None

        for csv in files:
            csv_cell = join(path_processed_files, csv)
            result_text_merge.insert(
                'end',
                f"Location of the merged .csv:\n {csv_cell} -> merged :) \n"
            )  # This prints the location and name of the merged .csv
            df = read_csv(csv_cell)

            if total is None:
                total = df
            else:
                total = concat([total, df], ignore_index=True)

        csv_name2 = csv_name + ".csv"
        csv = join(path_processed_files, csv_name2)
        total.to_csv(csv, index=False)

        # Save the arguments to the configuration file
        config['LastArgsMerge'] = {
            'path_processed_files': path_processed_files,
            'csv_name': csv_name,
            'pattern_merge': pattern
        }
        with open('last_arguments_process.ini', 'w') as configfile:
            config.write(configfile)

        result_text_merge.configure(
            state='disabled'
        )  #so that the user cannot edit the results window.

    def detect_horsetail(path, csv_filename, speed_stop_threshold):
        '''
        Detects the horsetail and stop values given the linear velocities. 

            Arguments:
            - path (string): the path where the files generated by the Fiji macro are located. 
            - csv_filename (string): the filename of the csv file to analyze.
            - speed_stop_threshold (float): the velocity value to detect the stop cell movement.
        '''
        speed_stop_threshold = float(speed_stop_threshold)
        
        result_text_horsetail.configure(state='normal')
        result_text_horsetail.delete('1.0', 'end')  # Clear the previous results
        
        # Read the csv and look for the column cal.speed
        try:
            df = read_csv(path + '/' + csv_filename + '.csv')
            velocity_column = 'cal.speed'
            
        except FileNotFoundError:
            result_text_horsetail.insert('end', 'The CSV file was not found.')
            return

        if velocity_column not in df.columns:
            result_text_horsetail.insert('end', f"The '{velocity_column}' column is not in the CSV; you must calculate the linear velocity first.")
            return

        # Loop
        particle = []
        durationHT = []
        durationPosHT = []
        position = 0

        for i in df['particle'].unique():
            cell = df[df['particle'] == i]
            a = []
            j = 0

            while sum(a) < 6:
                vector = cell[velocity_column][j:j + 6]
                a = [1 if v < speed_stop_threshold else 0 for v in vector]
                
                if vector.empty:
                    result_text_horsetail.insert('end',f"Particle: {i} stop not detected.\n \n")
                    break
                
                j += 1

            durationHT.append(j + 1)
            durationPosHT.append(len(cell['frame']) - j - 1)
            particle.append(i)
            position += 1

        # Horsetail values file
        data = DataFrame({
            'particle': particle,
            'durationHT': durationHT,
            'durationPosHT': durationPosHT
        })

        # Incorrect particle stop processing
        data.loc[data['durationPosHT'] == 3, 'durationHT'] += 3
        data.loc[data['durationPosHT'] == 3, 'durationPosHT'] = 0

        # Save final data
        data.to_csv(path + '/' + csv_filename + '_stop.csv', index=False)

        # Read the csv file processed with chromo
        df = read_csv(path + '/' + csv_filename + '.csv')

        # Read the file generated in the previous step
        df_stop_values = read_csv(path + '/' + csv_filename + '_stop.csv')

        vector_stop_values = []
        for i in df_stop_values['particle'].unique():
            vector_stop_values.extend(
                df_stop_values.loc[df_stop_values['particle'] == i,
                                   'durationPosHT'])

        df_subset = df.copy()

        all_cells = []
        for i in df_subset['particle'].unique():
            all_cells.extend(df_subset.loc[df_subset['particle'] == i,
                                           'particle'].unique())

        # Delete the posthorsetail phase
        n = 0
        out_df = df_subset.copy()

        for i in df['particle'].unique():
            n += 1
            out_df = out_df[~((out_df['frame'] > -vector_stop_values[n - 1]) &
                              (out_df['particle'] == i))]

        out_df.to_csv(path + '/' + csv_filename + '_horsetail.csv', index=False)
        
        # Display the path of the files.
        pattern = csv_filename
        files = [
            f for f in listdir(path)
            if f.lower().endswith(".csv") and pattern.lower() in f.lower()
        ]

        for csv in files:
            csv_cell = join(path, csv)
            result_text_horsetail.insert(
                    'end',
                    f"Location of the file:\n {csv_cell} \n"
                ) 

        # Save the arguments to the configuration file
        config['LastArgsHorsetail'] = {
            'path': path,
            'csv_filename': csv_filename,
            'speed_stop_threshold': speed_stop_threshold
        }
        with open('last_arguments_process.ini', 'w') as configfile:
            config.write(configfile)

        result_text_horsetail.configure(
            state='disabled'
        )  #so that the user cannot edit the results window.
        
    # Read the last arguments from the configuration file
    last_path = config.get('LastArgs', 'path', fallback='')
    last_save_path = config.get('LastArgs', 'save_path', fallback='')
    last_var_label = config.get('LastArgs', 'var_label', fallback='')
    last_pattern_process = config.get('LastArgs', 'pattern', fallback='')

    # Create the main window
    root = Tk()
    root.geometry("1100x900")
    root.title("Fiji Files Processor, Merger and Horsetail Detector")
    root.configure(bg="#95C8F3")

    # Configure the grid to adjust widget positions
    root.grid_columnconfigure(index=15, weight=1)
    root.grid_rowconfigure(index=11, weight=1)

    # Create labels and entry fields for input
    button_font = font.Font(family="Comic Sans MS", size=10)

    Label(root, text="Path:", font=button_font, bg="#95C8F3").grid(row=0,
                                                                   column=0,
                                                                   sticky='e')
    path_entry = Entry(root)
    path_entry.grid(row=0, column=1, sticky="we")
    Button(root,
           text="Browse",
           command=browse_folder_path,
           bg='#AEB5FF',
           fg='black',
           font=button_font).grid(row=0, column=2, sticky='we')

    Label(root, text="Save Path:", font=button_font,
          bg="#95C8F3").grid(row=1, column=0, sticky='e')
    save_path_entry = Entry(root)
    save_path_entry.grid(row=1, column=1, sticky="we")
    Button(root,
           text="Browse",
           command=browse_folder_save,
           bg='#AEB5FF',
           fg='black',
           font=button_font).grid(row=1, column=2, sticky='we')

    Label(root, text="Variable Label:", font=button_font,
          bg="#95C8F3").grid(row=2, column=0, sticky='e')
    var_label_entry = Entry(root)
    var_label_entry.grid(row=2, column=1, sticky='we')

    Label(root, text="Pattern:", font=button_font,
          bg="#95C8F3").grid(row=3, column=0, sticky='e')
    pattern_entry = Entry(root)
    pattern_entry.grid(row=3, column=1, sticky='we')

    root.grid_columnconfigure(1, weight=5)

    # Set the last arguments as default values in the entry fields
    path_entry.insert('end', last_path)
    save_path_entry.insert('end', last_save_path)
    var_label_entry.insert('end', last_var_label)
    if not last_pattern_process:
        pattern_entry.insert('end', "Results") #default value
    else:
        pattern_entry.insert('end', last_pattern_process)

    # Create a variable to store the checkbox state
    cal_speed_state = BooleanVar()
    cal_speed_state.set(False)  # Set the initial state to unchecked

    # Function to toggle the checkbox state
    def toggle_cal_speed():
        cal_speed_state.get() == True

    # Create the checkbox
    checkbox = Checkbutton(root,
                           text='Linear Velocity',
                           variable=cal_speed_state,
                           onvalue=True,
                           offvalue=False,
                           command=toggle_cal_speed,
                           bg='#AEB5FF',
                           font=button_font)
    checkbox.grid(row=4, column=0, columnspan=3, sticky='w')

    # Create a variable to store the checkbox state
    normalize_state = BooleanVar()
    normalize_state.set(False)  # Set the initial state to unchecked

    # Function to toggle the checkbox state
    def toggle_normalize():
        normalize_state.get() == True

    # Create the checkbox
    checkbox = Checkbutton(root,
                           text='Time Normalization',
                           variable=normalize_state,
                           onvalue=True,
                           offvalue=False,
                           command=toggle_normalize,
                           bg='#AEB5FF',
                           font=button_font)
    checkbox.grid(row=5, column=0, columnspan=3, sticky='w')

    # Create a button to process the files
    process_button = Button(
        root,
        text="Process Files",
        command=lambda: process_files(path_entry.get(), save_path_entry.get(
        ), var_label_entry.get(), pattern_entry.get(), cal_speed_state.get(),
                                      normalize_state.get()),
        font=font.Font(family="Comic Sans MS", size=11, weight="bold"),
        bg='#7DE198',
        fg='black')
    process_button.grid(row=6, column=0, columnspan=3, sticky='we')

    # Create a text widget to display the results of the processing.
    results_font = font.Font(family="MS Serif", size=11)
    result_text = Text(root, height=10, width=50, font=results_font)
    result_text.grid(row=7, column=0, columnspan=3, sticky='nsew')
    result_text.configure(bg="#81E3E1", fg="black", state="normal")

    # Read the last merge arguments from the configuration file
    last_path_processed_files = config.get('LastArgsMerge',
                                           'path_processed_files',
                                           fallback='')
    last_csv_name = config.get('LastArgsMerge', 'csv_name', fallback='')
    last_pattern_merge = config.get('LastArgsMerge',
                                    'pattern_merge',
                                    fallback='')

    # Create labels and entry fields for input of the merging function.
    button_font = font.Font(family="Comic Sans MS", size=10)

    Label(root, text="Merged CSV name:", font=button_font,
          bg="#95C8F3").grid(row=8, column=0, sticky='e')
    csv_name_entry = Entry(root)
    csv_name_entry.grid(row=8, column=1, sticky='we')

    Label(root, text="Pattern:", font=button_font,
          bg="#95C8F3").grid(row=9, column=0, sticky='e')
    pattern_merge_entry = Entry(root)
    pattern_merge_entry.grid(row=9, column=1, sticky='we')

    root.grid_columnconfigure(1, weight=5)

    # Set the last arguments as default values in the merge entry fields
    csv_name_entry.insert('end', last_csv_name)
    pattern_merge_entry.insert('end', last_pattern_merge)

    # Create a button to merge the files
    merge_button = Button(
        root,
        text="Merge Files",
        command=lambda: merge_files(save_path_entry.get(), csv_name_entry.get(
        ), pattern_merge_entry.get()),
        font=font.Font(family="Comic Sans MS", size=11, weight="bold"),
        bg='#7DE198',
        fg='black')
    merge_button.grid(row=10, column=0, columnspan=3, sticky='we')

    # Create a text widget to display the results of the merge.
    result_text_merge = Text(root, height=10, width=50, font=results_font)
    result_text_merge.grid(row=11, column=0, columnspan=3, sticky='nsew')
    result_text_merge.configure(bg="#81E3E1", fg="black", state="normal")
    
    # Read the last horsetail arguments from the configuration file
    last_path_horsetail = config.get('LastArgsHorsetail',
                                           'path',
                                           fallback='')
    last_name_horsetail = config.get('LastArgsHorsetail', 'csv_filename', fallback='')
    last_threshold = config.get('LastArgsHorsetail',
                                    'speed_stop_threshold',
                                    fallback='') 

    # Create labels and entry fields for input
    button_font = font.Font(family="Comic Sans MS", size=10)

    Label(root, text="Name of the CSV:", font=button_font,
          bg="#95C8F3").grid(row=4, column=14, sticky='e')
    csv_horsetail_entry = Entry(root)
    csv_horsetail_entry.grid(row=4, column=15, sticky='we')

    Label(root, text="Speed stop threshold:", font=button_font,
          bg="#95C8F3").grid(row=5, column=14, sticky='e')
    threshold_entry = Entry(root)
    threshold_entry.grid(row=5, column=15, sticky='we')

    root.grid_columnconfigure(1, weight=5)

    # Set the last arguments as default values in the entry fields
    csv_horsetail_entry.insert('end', last_name_horsetail)
    if not last_threshold:
        threshold_entry.insert('end', 0.2992296) #default value
    else:
        threshold_entry.insert('end', last_threshold)

    # Create a button to detect the horsetail and stop values
    horsetail_button = Button(
        root,
        text="Detect Horsetail",
        command=lambda: detect_horsetail(save_path_entry.get(), csv_horsetail_entry.get(
        ), float(threshold_entry.get())),
        font=font.Font(family="Comic Sans MS", size=11, weight="bold"),
        bg='#7DE198',
        fg='black')
    horsetail_button.grid(row=6, column=14, columnspan=3, sticky='we')

    # Create a text widget to display the results
    result_text_horsetail = Text(root, height=10, width=50, font=results_font)
    result_text_horsetail.grid(row=7, column=14, columnspan=3, rowspan=10,sticky='nsew')
    result_text_horsetail.configure(bg="#81E3E1", fg="black", state="normal")
    
    root.mainloop()


# Run the function to start the application
process_for_ChroMo()